# Solver configuration and error control
Author: Jørgen S. Dokken

In this section, we will go through how to specify what linear algebra solver we would like to use to solve our PDEs, as well as how to verify the implemenation by considering convergence rates.

```{math}
-\Delta u &= f &&\text{in } \Omega\\
u&= u_D &&\text{on } \partial \Omega.
```
Using the manufactured solution $u_D=\cos(2\pi x)\cos(2\pi y)$, we obtain $f=8\pi^2\cos(2\pi x)\cos(2\pi y)$.
We start by creating a generic module for evaluating the analytical solution  at any point $x$.

In [7]:
import dolfinx
import numpy
import ufl

def u_ex(mod):
    return lambda x: mod.cos(2*mod.pi*x[0])*mod.cos(2*mod.pi*x[1])

Note that the return type of `u_ex` is a `lambda` function. Thus, we can create two different lambda functions, one using `numpy` (which will be used for interpolation) and one using `ufl` (which will be used for defining the source term)

In [8]:
u_numpy = u_ex(numpy)
u_ufl = u_ex(ufl)

We start by using ufl to define our source term, using `ufl.SpatialCoordinate` as input to `u_ufl`.

In [9]:
from mpi4py import MPI
mesh = dolfinx.UnitSquareMesh(MPI.COMM_WORLD, 30, 30)
x = ufl.SpatialCoordinate(mesh)
f = -ufl.div(ufl.grad(u_ufl(x)))

Next, we define our linear variational problem

In [10]:
import petsc4py, sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
V = dolfinx.FunctionSpace(mesh, ("CG", 1))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = u * v * ufl.dx
L = f * v * ufl.dx
u_bc = dolfinx.Function(V)
u_bc.interpolate(u_numpy)
u_bc.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim -1, lambda x: numpy.full(x.shape[1], True))
dofs = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim-1, facets)
bcs = [dolfinx.DirichletBC(u_bc, dofs)]

We start by solving the problem with an LU factorization, a direct solver method.

In [11]:

default_problem = dolfinx.fem.LinearProblem(a, L, bcs=bcs,
petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = default_problem.solve()

We now look at the solver process by inspecting the `PETSc`-solver. As the view-options in PETSc are not adjusted for notebooks (`solver.view()` will print output to the terminal if used in a `.py` file), we write the solver output to file and read it in and print the output.

In [12]:
solver = default_problem.solver
viewer = PETSc.Viewer().createASCII("solver_output.txt")
solver.view(viewer)
solver_output = open("solver_output.txt", "r")
for line in solver_output.readlines():
    print(line)

KSP Object: (dolfinx_solve_140163524346160) 1 MPI processes

  type: preonly

  maximum iterations=10000, initial guess is zero

  tolerances:  relative=1e-05, absolute=1e-50, divergence=10000.

  left preconditioning

  using NONE norm type for convergence test

PC Object: (dolfinx_solve_140163524346160) 1 MPI processes

  type: lu

    out-of-place factorization

    tolerance for zero pivot 2.22045e-14

    matrix ordering: nd

    factor fill ratio given 5., needed 5.0719

      Factored matrix follows:

        Mat Object: 1 MPI processes

          type: seqaij

          rows=961, cols=961

          package used to perform factorization: petsc

          total: nonzeros=32871, allocated nonzeros=32871

            not using I-node routines

  linear system matrix = precond matrix:

  Mat Object: 1 MPI processes

    type: seqaij

    rows=961, cols=961

    total: nonzeros=6481, allocated nonzeros=6481

    total number of mallocs used during MatSetValues calls=0

      not usi